# 1.1 Install Python Packages 

In [1]:
pip install pyshp

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import shapefile  # "pip install pyship" 
import folium     # "pip install folium"
import glob
import pandas as pd
from collections import OrderedDict

# 1.2 Import Dataset

In [4]:
shp_files = glob.glob('./**/*.shp', recursive=True)
dbf_files = glob.glob('./**/*.dbf', recursive=True)

# Take a look at the top 8 .shp files
print(shp_files[:8])

['.\\MRTLRTStnPtt.shp']


In [5]:
query = 'LRT'
index = None
for i, (shp, dbf) in enumerate(zip(shp_files, dbf_files)):
    if query.lower() in shp.lower():
        index = i
        break
print('The query \"{0}\" is at index {1}'.format(query, index))

The query "LRT" is at index 0


In [6]:
myshp = open(shp_files[index], 'rb')
mydbf = open(dbf_files[index], 'rb')
sf = shapefile.Reader(shp=myshp, dbf=mydbf)
records = sf.shapeRecords()

In [7]:
# Count the number of shape objects
print('There are {} shape objects'.format(len(records)))

There are 185 shape objects


In [8]:
# Look at top 5 items
for x in records[:5]:
    msg = '{0:<23} ({1:<1})  @  {2}'
    print(msg.format(x.record[1], x.record[2], x.shape.points[0]))

EUNOS MRT STATION       (EW7)  @  [35782.955299999565, 33560.077600000426]
CHINESE GARDEN MRT STATION (EW25)  @  [16790.746600000188, 36056.30189999938]
KHATIB MRT STATION      (NS14)  @  [27962.310800000094, 44352.56799999997]
KRANJI MRT STATION      (NS7)  @  [20081.697399999946, 45214.54790000059]
REDHILL MRT STATION     (EW18)  @  [26163.47800000012, 30218.819599999115]


In [9]:
# Retrieve the "STN_NAME", "STN_CODE", and the coordinates
list_of_tuples = [(x.record[1], x.record[2], x.shape.points[0]) for x in records]

# Unzip list of tuples into individual lists
stn_names, stn_codes, coordinates = zip(*list_of_tuples)

d = OrderedDict([('stn_names', stn_names), 
                 ('stn_codes', stn_codes), 
                 ('coordinates', coordinates)])

df = pd.DataFrame(d)
df

,stn_names,stn_codes,coordinates
0,EUNOS MRT STATION,EW7,"[35782.955299999565, 33560.077600000426]"
1,CHINESE GARDEN MRT STATION,EW25,"[16790.746600000188, 36056.30189999938]"
2,KHATIB MRT STATION,NS14,"[27962.310800000094, 44352.56799999997]"
3,KRANJI MRT STATION,NS7,"[20081.697399999946, 45214.54790000059]"
4,REDHILL MRT STATION,EW18,"[26163.47800000012, 30218.819599999115]"
...,...,...,...
180,TELOK BLANGAH MRT STATION,CC28,"[25376.846900000237, 28138.96670000069]"
181,TELOK AYER MRT STATION,DT18,"[29667.621600000188, 29414.58669999987]"
182,NEWTON MRT STATION,NS21,"[28519.36160000041, 32735.207200000063]"
183,CANBERRA MRT STATION,NS12,"[27598.621771013826, 47200.24193103178]"


# 1.3 Fixing Coordinates

In [10]:
# Singapore has a special coordinate system called SVY21
# Use open source code from cgcai to fix the coordinates 
# %load "https://gist.githubusercontent.com/jovianlin/30b0fd93e2b835cb1e8d93bfa0e1e62f/raw/89548360fb54ab1e0441b0935fc52fbbac6d263e/SVY21.py"
#! /usr/bin/python

import math

class SVY21:
    # Ref: http://www.linz.govt.nz/geodetic/conversion-coordinates/projection-conversions/transverse-mercator-preliminary-computations/index.aspx
    
    # WGS84 Datum
    a = 6378137
    f = 1 / 298.257223563

    # SVY21 Projection
    # Fundamental point: Base 7 at Pierce Resevoir.
    # Latitude: 1 22 02.9154 N, longitude: 103 49 31.9752 E (of Greenwich).

    # Known Issue: Setting (oLat, oLon) to the exact coordinates specified above
    # results in computation being slightly off. The values below give the most 
    # accurate represenation of test data.
    oLat = 1.366666     # origin's lat in degrees
    oLon = 103.833333   # origin's lon in degrees
    oN = 38744.572      # false Northing
    oE = 28001.642      # false Easting
    k = 1               # scale factor

    def __init__(self):
        self.b = self.a * (1 - self.f)
        self.e2 = (2 * self.f) - (self.f * self.f)
        self.e4 = self.e2 * self.e2
        self.e6 = self.e4 * self.e2
        self.A0 = 1 - (self.e2 / 4) - (3 * self.e4 / 64) - (5 * self.e6 / 256);
        self.A2 = (3. / 8.) * (self.e2 + (self.e4 / 4) + (15 * self.e6 / 128));
        self.A4 = (15. / 256.) * (self.e4 + (3 * self.e6 / 4));
        self.A6 = 35 * self.e6 / 3072;

    def computeSVY21(self, lat, lon):
        """
        Returns a pair (N, E) representing Northings and Eastings in SVY21.
        """

        latR = lat * math.pi / 180
        sinLat = math.sin(latR)
        sin2Lat = sinLat * sinLat
        cosLat = math.cos(latR)
        cos2Lat = cosLat * cosLat
        cos3Lat = cos2Lat * cosLat
        cos4Lat = cos3Lat * cosLat
        cos5Lat = cos4Lat * cosLat
        cos6Lat = cos5Lat * cosLat
        cos7Lat = cos6Lat * cosLat

        rho = self.calcRho(sin2Lat)
        v = self.calcV(sin2Lat)
        psi = v / rho
        t = math.tan(latR)
        w = (lon - self.oLon) * math.pi / 180

        M = self.calcM(lat)
        Mo = self.calcM(self.oLat)

        w2 = w * w
        w4 = w2 * w2
        w6 = w4 * w2
        w8 = w6 * w2

        psi2 = psi * psi
        psi3 = psi2 * psi
        psi4 = psi3 * psi

        t2 = t * t
        t4 = t2 * t2
        t6 = t4 * t2

        # Compute Northing
        nTerm1 = w2 / 2 * v * sinLat * cosLat
        nTerm2 = w4 / 24 * v * sinLat * cos3Lat * (4 * psi2 + psi - t2)
        nTerm3 = w6 / 720 * v * sinLat * cos5Lat * ((8 * psi4) * (11 - 24 * t2) - (28 * psi3) * (1 - 6 * t2) + psi2 * (1 - 32 * t2) - psi * 2 * t2 + t4)
        nTerm4 = w8 / 40320 * v * sinLat * cos7Lat * (1385 - 3111 * t2 + 543 * t4 - t6)
        N = self.oN + self.k * (M - Mo + nTerm1 + nTerm2 + nTerm3 + nTerm4)

        # Compute Easting
        eTerm1 = w2 / 6 * cos2Lat * (psi - t2)
        eTerm2 = w4 / 120 * cos4Lat * ((4 * psi3) * (1 - 6 * t2) + psi2 * (1 + 8 * t2) - psi * 2 * t2 + t4)
        eTerm3 = w6 / 5040 * cos6Lat * (61 - 479 * t2 + 179 * t4 - t6)
        E = self.oE + self.k * v * w * cosLat * (1 + eTerm1 + eTerm2 + eTerm3)

        return (N, E)

    def calcM(self, lat):
        latR = lat * math.pi / 180
        return self.a * ((self.A0 * latR) - (self.A2 * math.sin(2 * latR)) + (self.A4 * math.sin(4 * latR)) - (self.A6 * math.sin(6 * latR)))

    def calcRho(self, sin2Lat):
        num = self.a * (1 - self.e2)
        denom = math.pow(1 - self.e2 * sin2Lat, 3. / 2.)
        return num / denom

    def calcV(self, sin2Lat):
        poly = 1 - self.e2 * sin2Lat
        return self.a / math.sqrt(poly)

    def computeLatLon(self, N, E):
        """
        Returns a pair (lat, lon) representing Latitude and Longitude.
        """

        Nprime = N - self.oN
        Mo = self.calcM(self.oLat)
        Mprime = Mo + (Nprime / self.k)
        n = (self.a - self.b) / (self.a + self.b)
        n2 = n * n
        n3 = n2 * n
        n4 = n2 * n2
        G = self.a * (1 - n) * (1 - n2) * (1 + (9 * n2 / 4) + (225 * n4 / 64)) * (math.pi / 180)
        sigma = (Mprime * math.pi) / (180. * G)
        
        latPrimeT1 = ((3 * n / 2) - (27 * n3 / 32)) * math.sin(2 * sigma)
        latPrimeT2 = ((21 * n2 / 16) - (55 * n4 / 32)) * math.sin(4 * sigma)
        latPrimeT3 = (151 * n3 / 96) * math.sin(6 * sigma)
        latPrimeT4 = (1097 * n4 / 512) * math.sin(8 * sigma)
        latPrime = sigma + latPrimeT1 + latPrimeT2 + latPrimeT3 + latPrimeT4

        sinLatPrime = math.sin(latPrime)
        sin2LatPrime = sinLatPrime * sinLatPrime

        rhoPrime = self.calcRho(sin2LatPrime)
        vPrime = self.calcV(sin2LatPrime)
        psiPrime = vPrime / rhoPrime
        psiPrime2 = psiPrime * psiPrime
        psiPrime3 = psiPrime2 * psiPrime
        psiPrime4 = psiPrime3 * psiPrime
        tPrime = math.tan(latPrime)
        tPrime2 = tPrime * tPrime
        tPrime4 = tPrime2 * tPrime2
        tPrime6 = tPrime4 * tPrime2
        Eprime = E - self.oE
        x = Eprime / (self.k * vPrime)
        x2 = x * x
        x3 = x2 * x
        x5 = x3 * x2
        x7 = x5 * x2

        # Compute Latitude
        latFactor = tPrime / (self.k * rhoPrime)
        latTerm1 = latFactor * ((Eprime * x) / 2)
        latTerm2 = latFactor * ((Eprime * x3) / 24) * ((-4 * psiPrime2) + (9 * psiPrime) * (1 - tPrime2) + (12 * tPrime2))
        latTerm3 = latFactor * ((Eprime * x5) / 720) * ((8 * psiPrime4) * (11 - 24 * tPrime2) - (12 * psiPrime3) * (21 - 71 * tPrime2) + (15 * psiPrime2) * (15 - 98 * tPrime2 + 15 * tPrime4) + (180 * psiPrime) * (5 * tPrime2 - 3 * tPrime4) + 360 * tPrime4)
        latTerm4 = latFactor * ((Eprime * x7) / 40320) * (1385 - 3633 * tPrime2 + 4095 * tPrime4 + 1575 * tPrime6)
        lat = latPrime - latTerm1 + latTerm2 - latTerm3 + latTerm4

        # Compute Longitude
        secLatPrime = 1. / math.cos(lat)
        lonTerm1 = x * secLatPrime
        lonTerm2 = ((x3 * secLatPrime) / 6) * (psiPrime + 2 * tPrime2)
        lonTerm3 = ((x5 * secLatPrime) / 120) * ((-4 * psiPrime3) * (1 - 6 * tPrime2) + psiPrime2 * (9 - 68 * tPrime2) + 72 * psiPrime * tPrime2 + 24 * tPrime4)
        lonTerm4 = ((x7 * secLatPrime) / 5040) * (61 + 662 * tPrime2 + 1320 * tPrime4 + 720 * tPrime6)
        lon = (self.oLon * math.pi / 180) + lonTerm1 - lonTerm2 + lonTerm3 - lonTerm4

        return (lat / (math.pi / 180), lon / (math.pi / 180))

In [11]:
# Run the code above
svy = SVY21()
df['new_coordinates'] = df.coordinates.apply(lambda x: svy.computeLatLon(x[1], x[0]))
df.head()

,stn_names,stn_codes,coordinates,new_coordinates
0,EUNOS MRT STATION,EW7,"[35782.955299999565, 33560.077600000426]","(1.3197782848869883, 103.90325213335886)"
1,CHINESE GARDEN MRT STATION,EW25,"[16790.746600000188, 36056.30189999938]","(1.3423521542087522, 103.73259640485367)"
2,KHATIB MRT STATION,NS14,"[27962.310800000094, 44352.56799999997]","(1.4173827034869275, 103.83297957440541)"
3,KRANJI MRT STATION,NS7,"[20081.697399999946, 45214.54790000059]","(1.4251770321043908, 103.76216507762184)"
4,REDHILL MRT STATION,EW18,"[26163.47800000012, 30218.819599999115]","(1.2895620597351773, 103.81681633682066)"


In [12]:
# drop coordinates column
df = df.drop(['coordinates'], axis=1)
df.head()

,stn_names,stn_codes,new_coordinates
0,EUNOS MRT STATION,EW7,"(1.3197782848869883, 103.90325213335886)"
1,CHINESE GARDEN MRT STATION,EW25,"(1.3423521542087522, 103.73259640485367)"
2,KHATIB MRT STATION,NS14,"(1.4173827034869275, 103.83297957440541)"
3,KRANJI MRT STATION,NS7,"(1.4251770321043908, 103.76216507762184)"
4,REDHILL MRT STATION,EW18,"(1.2895620597351773, 103.81681633682066)"


In [13]:
# rename column
df.rename(columns={'new_coordinates': 'coordinates'}, inplace=True)
df.head()

,stn_names,stn_codes,coordinates
0,EUNOS MRT STATION,EW7,"(1.3197782848869883, 103.90325213335886)"
1,CHINESE GARDEN MRT STATION,EW25,"(1.3423521542087522, 103.73259640485367)"
2,KHATIB MRT STATION,NS14,"(1.4173827034869275, 103.83297957440541)"
3,KRANJI MRT STATION,NS7,"(1.4251770321043908, 103.76216507762184)"
4,REDHILL MRT STATION,EW18,"(1.2895620597351773, 103.81681633682066)"


In [14]:
#split coordinates into latitude and longitude
df[['latitude', 'longitude']] = pd.DataFrame(df['coordinates'].tolist(), index=df.index)                                                                                                                       
df.head()                                                                                                                                                                     

,stn_names,stn_codes,coordinates,latitude,longitude
0,EUNOS MRT STATION,EW7,"(1.3197782848869883, 103.90325213335886)",1.319778,103.903252
1,CHINESE GARDEN MRT STATION,EW25,"(1.3423521542087522, 103.73259640485367)",1.342352,103.732596
2,KHATIB MRT STATION,NS14,"(1.4173827034869275, 103.83297957440541)",1.417383,103.832980
3,KRANJI MRT STATION,NS7,"(1.4251770321043908, 103.76216507762184)",1.425177,103.762165
4,REDHILL MRT STATION,EW18,"(1.2895620597351773, 103.81681633682066)",1.289562,103.816816


In [15]:
# drop coordinates column
df = df.drop(['coordinates'], axis=1)
df.head()

,stn_names,stn_codes,latitude,longitude
0,EUNOS MRT STATION,EW7,1.319778,103.903252
1,CHINESE GARDEN MRT STATION,EW25,1.342352,103.732596
2,KHATIB MRT STATION,NS14,1.417383,103.832980
3,KRANJI MRT STATION,NS7,1.425177,103.762165
4,REDHILL MRT STATION,EW18,1.289562,103.816816


# 1.4 Visualise Train Stations

In [16]:
#Download dependencies
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [17]:
# Pass coordinate of Singapore

latitude = 1.3521
longitude = 103.8198

In [18]:
#create Singapore map
map_sg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['latitude'], df['longitude'], df['stn_names']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg

# 2.1 Explore with Foursquare API

In [19]:
#Define foursq credentials and version
CLIENT_ID = 'HMGEIRXXKYNJA5O5TMNUAJDU3J4UFWUL5Y5OT' # your Foursquare ID
CLIENT_SECRET = 'TLUIWRS1O2TCR5M4UHF1MUGCBHSKKG0MM' # your Foursquare Secret
VERSION = '20200130' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HMGEIRXXKYNJA5O5TMNUAJDU3J4UFWUL5Y5OTQPJPF2EPQGN
CLIENT_SECRET:TLUIWRS1O2TCR5M4UHF1MUGCBHSKKG0MMAHMX1EMFZQWYDNT


In [22]:
#get the top 50 venues that are in each neighborhood within a radius of 400 meters.
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 400 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['stn_names', 
                  'stn_names latitude', 
                  'stn_names longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#run getNearbyVenues
stn_venues = getNearbyVenues(names=df['stn_names'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

EUNOS MRT STATION
CHINESE GARDEN MRT STATION
KHATIB MRT STATION
KRANJI MRT STATION
REDHILL MRT STATION
YEW TEE MRT STATION
PIONEER MRT STATION
COMMONWEALTH MRT STATION
PAYA LEBAR MRT STATION
SIMEI MRT STATION
TAMPINES MRT STATION
ADMIRALTY MRT STATION
YISHUN MRT STATION
PASIR RIS MRT STATION
CLEMENTI MRT STATION
ALJUNIED MRT STATION
ANG MO KIO MRT STATION
EXPO MRT STATION
CHOA CHU KANG MRT STATION
JURONG EAST MRT STATION
QUEENSTOWN MRT STATION
TUAS LINK MRT STATION
GUL CIRCLE MRT STATION
WOODLANDS MRT STATION
TUAS CRESCENT MRT STATION
KALLANG MRT STATION
TUAS WEST ROAD MRT STATION
BUONA VISTA MRT STATION
DOVER MRT STATION
SEMBAWANG MRT STATION
KEMBANGAN MRT STATION
BUKIT BATOK MRT STATION
ANG MO KIO MRT STATION
BOON LAY MRT STATION
BUKIT GOMBAK MRT STATION
MARSILING MRT STATION
BEDOK MRT STATION
JOO KOON MRT STATION
LAKESIDE MRT STATION
TANAH MERAH MRT STATION
YIO CHU KANG MRT STATION
OASIS LRT STATION
SAM KEE LRT STATION
FARMWAY LRT STATION
KUPANG LRT STATION
RENJONG LRT STATION
BUKIT

In [43]:
# check size of df
print(stn_venues.shape)
stn_venues.head()

(5177, 7)


,stn_names,stn_names latitude,stn_names longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,EUNOS MRT STATION,1.319778,103.903252,福春叻沙卤面 Laksa & Lor Mee,1.320254,103.904132,Noodle House
1,EUNOS MRT STATION,1.319778,103.903252,GYMM BOXX,1.317912,103.901094,Gym
2,EUNOS MRT STATION,1.319778,103.903252,Mr Bean,1.319703,103.902479,Breakfast Spot
3,EUNOS MRT STATION,1.319778,103.903252,香港街珍達記,1.316703,103.901415,Chinese Restaurant
4,EUNOS MRT STATION,1.319778,103.903252,LiHO,1.319729,103.902512,Bubble Tea Shop


In [44]:
 #check how many venues were returned for each neighborhood
stn_venues.groupby('stn_names').count()

,stn_names latitude,stn_names longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
stn_names,,,,,,
ADMIRALTY MRT STATION,9,9,9,9,9,9
ALJUNIED MRT STATION,50,50,50,50,50,50
ANG MO KIO MRT STATION,80,80,80,80,80,80
BAKAU LRT STATION,10,10,10,10,10,10
BANGKIT LRT STATION,9,9,9,9,9,9
BARTLEY MRT STATION,12,12,12,12,12,12
BAYFRONT MRT STATION,50,50,50,50,50,50
BEAUTY WORLD MRT STATION,50,50,50,50,50,50
BEDOK MRT STATION,50,50,50,50,50,50


In [45]:
# check how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(stn_venues['Venue Category'].unique())))

There are 319 uniques categories.


# 2.2 Analyze each station

In [46]:
# one hot encoding

stn_onehot = pd.get_dummies(stn_venues[['Venue Category']], prefix="", prefix_sep="")
# add stn_names column back to dataframe

stn_onehot['stn_names'] = stn_venues['stn_names'] 

# move stn_names column to the first column
cols = stn_onehot.columns.tolist()
n = int(cols.index('stn_names'))
cols = [cols[n]] + cols[:n] + cols[n+1:]
stn_onehot = stn_onehot[cols]

stn_onehot.head()

,stn_names,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lottery Retailer,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Sup

In [47]:
stn_onehot.shape

(5177, 320)

In [48]:
# group rows by stn_names and by taking the mean of the frequency of occurrence of each category
stn_grouped = stn_onehot.groupby('stn_names').mean().reset_index()
stn_grouped

,stn_names,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bay,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lottery Retailer,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Sup

In [49]:
stn_grouped.shape

(165, 320)

In [50]:
# print each stn_names along with the top 5 most common venue
num_top_venues = 5

for hood in stn_grouped['stn_names']:
    print("----"+hood+"----")
    temp = stn_grouped[stn_grouped['stn_names'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ADMIRALTY MRT STATION----
         venue  freq
0  Supermarket  0.22
1       Bakery  0.22
2   Food Court  0.11
3  Bus Station  0.11
4        Plaza  0.11


----ALJUNIED MRT STATION----
                venue  freq
0    Asian Restaurant  0.10
1  Chinese Restaurant  0.10
2  Dim Sum Restaurant  0.08
3        Noodle House  0.08
4         Coffee Shop  0.06


----ANG MO KIO MRT STATION----
                 venue  freq
0          Coffee Shop  0.10
1         Dessert Shop  0.08
2           Food Court  0.08
3  Japanese Restaurant  0.05
4      Bubble Tea Shop  0.05


----BAKAU LRT STATION----
                 venue  freq
0         Dessert Shop   0.1
1  Japanese Restaurant   0.1
2          Supermarket   0.1
3           Shoe Store   0.1
4          Bus Station   0.1


----BANGKIT LRT STATION----
                     venue  freq
0               Food Court  0.22
1             Noodle House  0.11
2       Miscellaneous Shop  0.11
3  Fruit & Vegetable Store  0.11
4                   Market  0.11


----BA

In [51]:
# put into Panda frame

#def a function to sort venues in decending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
#create the new dataframe and display the top 10 venues for each stn_names
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['stn_names']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
stn_venues_sorted = pd.DataFrame(columns=columns)
stn_venues_sorted['stn_names'] = stn_grouped['stn_names']

for ind in np.arange(stn_grouped.shape[0]):
    stn_venues_sorted.iloc[ind, 1:] = return_most_common_venues(stn_grouped.iloc[ind, :], num_top_venues)

stn_venues_sorted.head()

,stn_names,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADMIRALTY MRT STATION,Bakery,Supermarket,Bus Station,Plaza,Food Court,Coffee Shop,Paper / Office Supplies Store,Food Truck,Food Stand,Food Service
1,ALJUNIED MRT STATION,Asian Restaurant,Chinese Restaurant,Noodle House,Dim Sum Restaurant,Coffee Shop,Food Court,Vegetarian / Vegan Restaurant,BBQ Joint,Seafood Restaurant,Farmers Market
2,ANG MO KIO MRT STATION,Coffee Shop,Dessert Shop,Food Court,Supermarket,Bubble Tea Shop,Japanese Restaurant,Snack Place,Miscellaneous Shop,Modern European Restaurant,Frozen Yogurt Shop
3,BAKAU LRT STATION,Basketball Court,Supermarket,Fast Food Restaurant,Trail,Sandwich Place,Malay Restaurant,Bus Station,Shoe Store,Japanese Restaurant,Dessert Shop
4,BANGKIT LRT STATION,Food Court,Fruit & Vegetable Store,Park,Noodle House,Bike Trail,Market,Miscellaneous Shop,Coffee Shop,Food & Drink Shop,Fishing Spot


In [70]:
stn_venues_sorted.shape

(165, 12)

# 3.1 Cluster the Stations

In [53]:
# Run k-means to cluster the stn_names into 6 clusters

# set number of clusters
kclusters = 5

stn_grouped_clustering = stn_grouped.drop('stn_names', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(stn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 0, 3, 3, 0, 3, 1])

In [54]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each stn_names

# add clustering labels
stn_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

stn_merged = df

# merge stn_grouped with df to add latitude/longitude for each neighborhood
stn_merged = stn_merged.join(stn_venues_sorted.set_index('stn_names'), on='stn_names')

stn_merged.head() # check the last columns!

,stn_names,stn_codes,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EUNOS MRT STATION,EW7,1.319778,103.903252,0.0,Chinese Restaurant,Noodle House,Coffee Shop,Seafood Restaurant,Gym,Bubble Tea Shop,Train Station,Vegetarian / Vegan Restaurant,Asian Restaurant,Food Court
1,CHINESE GARDEN MRT STATION,EW25,1.342352,103.732596,0.0,Chinese Restaurant,Coffee Shop,Asian Restaurant,Indian Restaurant,Train Station,Café,Food Court,Food,Filipino Restaurant,Fish & Chips Shop
2,KHATIB MRT STATION,NS14,1.417383,103.832980,1.0,Coffee Shop,Asian Restaurant,Supermarket,Food Court,Grocery Store,Park,Shopping Mall,Middle Eastern Restaurant,Train Station,Fast Food Restaurant
3,KRANJI MRT STATION,NS7,1.425177,103.762165,3.0,Stadium,Racetrack,Night Market,Bakery,Lighthouse,Bus Line,Go Kart Track,Food & Drink Shop,Fish & Chips Shop,Fishing Spot
4,REDHILL MRT STATION,EW18,1.289562,103.816816,0.0,Chinese Restaurant,Coffee Shop,Gym,Pool,Park,Hainan Restaurant,Train Station,Vegetarian / Vegan Restaurant,Malay Restaurant,Bus Station


In [59]:
stn_merged.shape

(185, 15)

In [61]:
stn_merged = stn_merged.dropna()

In [62]:
#stn_merged['Cluster Labels'] = stn_merged['Cluster Labels'].astype(np.int64)

C:\Users\Louie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# 3.2 Visualise Clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stn_merged['latitude'], stn_merged['longitude'], stn_merged['stn_names'], stn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 3.3 Examine Cluster

In [71]:
# Cluster 1
stn_merged.loc[stn_merged['Cluster Labels'] == 0, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,EW7,Chinese Restaurant,Noodle House,Coffee Shop,Seafood Restaurant,Gym,Bubble Tea Shop,Train Station,Vegetarian / Vegan Restaurant,Asian Restaurant,Food Court
1,EW25,Chinese Restaurant,Coffee Shop,Asian Restaurant,Indian Restaurant,Train Station,Café,Food Court,Food,Filipino Restaurant,Fish & Chips Shop
4,EW18,Chinese Restaurant,Coffee Shop,Gym,Pool,Park,Hainan Restaurant,Train Station,Vegetarian / Vegan Restaurant,Malay Restaurant,Bus Station
7,EW20,Chinese Restaurant,Bus Station,Noodle House,Asian Restaurant,Food Court,Garden,BBQ Joint,Malay Restaurant,Fast Food Restaurant,Fried Chicken Joint
12,NS13,Chinese Restaurant,Food Court,Coffee Shop,Park,Italian Restaurant,Fried Chicken Joint,Hainan Restaurant,Supermarket,Furniture / Home Store,Mexican Restaurant
15,EW9,Asian Restaurant,Chinese Restaurant,Noodle House,Dim Sum Restaurant,Coffee Shop,Food Court,Vegetarian / Vegan Restaurant,BBQ Joint,Seafood Restaurant,Farmers Market
20,EW19,Food Court,Chinese Restaurant,Noodle House,Train Station,Spa,Supermarket,Café,Seafood Restaurant,Stadium,Italian Restaurant
22,EW30,Food Court,Coffee Shop,Train Station,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop
25,EW10,Food Court,Hostel,Snack Place,BBQ Joint,Rock Club,Thai Restaurant,Supermarket,Historic Site,Noodle House,Soup Place
30,EW6,Chinese Restaurant,Indian Restaurant,Asian Restaurant,Shopping Mall,Seafood Restaurant,Bistro,Coffee Shop,Train Station,Food Court,Gastropub


In [65]:
# Cluster 2
stn_merged.loc[stn_merged['Cluster Labels'] == 1, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,NS14,Coffee Shop,Asian Restaurant,Supermarket,Food Court,Grocery Store,Park,Shopping Mall,Middle Eastern Restaurant,Train Station,Fast Food Restaurant
5,NS5,Fast Food Restaurant,Shopping Mall,Pharmacy,Japanese Restaurant,Diner,Sushi Restaurant,Food Court,Pool,Café,Sandwich Place
6,EW28,Stadium,Electronics Store,Supermarket,Bus Station,Fast Food Restaurant,Golf Course,Shopping Mall,Seafood Restaurant,Gym / Fitness Center,Sporting Goods Shop
8,EW8,Asian Restaurant,Fast Food Restaurant,Supermarket,Indonesian Restaurant,Hotpot Restaurant,Food Court,Dessert Shop,Coffee Shop,Shopping Mall,Snack Place
9,EW3,Coffee Shop,Bus Station,Japanese Restaurant,Chinese Restaurant,Convenience Store,Soup Place,Noodle House,Soccer Field,Food Court,Café
11,NS10,Bakery,Supermarket,Bus Station,Plaza,Food Court,Coffee Shop,Paper / Office Supplies Store,Food Truck,Food Stand,Food Service
13,EW1,Fast Food Restaurant,Food Court,Italian Restaurant,Sandwich Place,Coffee Shop,Bus Station,Snack Place,Café,Supermarket,Bakery
14,EW23,Coffee Shop,Food Court,Dessert Shop,Chinese Restaurant,Asian Restaurant,Dim Sum Restaurant,Fast Food Restaurant,Japanese Restaurant,Chinese Breakfast Place,Electronics Store
16,NS16,Coffee Shop,Dessert Shop,Food Court,Supermarket,Bubble Tea Shop,Japanese Restaurant,Snack Place,Miscellaneous Shop,Modern European Restaurant,Frozen Yogurt Shop
18,NS4,Fast Food Restaurant,Coffee Shop,Portuguese Restaurant,Snack Place,Thai Restaurant,Sandwich Place,Park,Noodle House,Café,Bakery


In [66]:
# Cluster 3
stn_merged.loc[stn_merged['Cluster Labels'] == 2, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
184,TE1,Rest Area,Indian Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop


In [67]:
# Cluster 4
stn_merged.loc[stn_merged['Cluster Labels'] == 3, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,NS7,Stadium,Racetrack,Night Market,Bakery,Lighthouse,Bus Line,Go Kart Track,Food & Drink Shop,Fish & Chips Shop,Fishing Spot
10,EW2,Café,Bakery,Sushi Restaurant,Coffee Shop,Gym,Thai Restaurant,Shopping Mall,Pharmacy,Clothing Store,Chinese Restaurant
17,CG1,Café,Coffee Shop,Japanese Restaurant,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Food Court,Sporting Goods Shop,Hotel,Filipino Restaurant
19,EW24 / NS1,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Department Store,Bubble Tea Shop,Korean Restaurant,Shopping Mall,Bakery,Clothing Store,Supermarket
21,EW33,Chinese Restaurant,Gastropub,Sports Club,Train Station,Harbor / Marina,Bus Station,Sporting Goods Shop,Food Truck,Fried Chicken Joint,Filipino Restaurant
23,NS9,Japanese Restaurant,Café,Shopping Mall,Clothing Store,Fast Food Restaurant,Frozen Yogurt Shop,Coffee Shop,Asian Restaurant,Chinese Restaurant,Electronics Store
24,EW31,Harbor / Marina,Train Station,Food Truck,Cafeteria,Yunnan Restaurant,Food,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market
27,EW21,Japanese Restaurant,Coffee Shop,Indian Restaurant,Food Court,Chinese Restaurant,Bakery,Shopping Mall,Salad Place,Café,Performing Arts Venue
36,EW5,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Asian Restaurant,Café,Food Court,Sushi Restaurant,Fast Food Restaurant,Supermarket,Sandwich Place
42,PW1,Chinese Restaurant,Café,Japanese Restaurant,Fast Food Restaurant,Steakhouse,Vegetarian / Vegan Restaurant,Sushi Restaurant,Park,Pizza Place,Clothing Store


In [68]:
# Cluster 5
stn_merged.loc[stn_merged['Cluster Labels'] == 4, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,SW7,Playground,Bus Station,Bookstore,Food Service,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop
78,SE1,Playground,Coffee Shop,Sculpture Garden,Dog Run,Food,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market


In [69]:
# Cluster 6
stn_merged.loc[stn_merged['Cluster Labels'] == 5, stn_merged.columns[[1] + list(range(5, stn_merged.shape[1]))]]

,stn_codes,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
